In [1]:
import numpy as np
from copy import deepcopy
from sklearn.datasets import load_boston
from sklearn.ensemble import VotingRegressor as skVotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
class VotingRegressor():
    def __init__(self, estimators):
        self.estimators = estimators

    def fit(self, X, y):
        self.estimators_ = [deepcopy(est).fit(X, y) for _, est in self.estimators]
        return self

    def transform(self, X):
        prob = np.array([est.predict(X) for est in self.estimators_]).T
        return prob

    def predict(self, X):
        prob = self.transform(X)
        return np.mean(prob, axis=1)

In [3]:
X, y = load_boston(return_X_y=True)
clf1 = LinearRegression()
clf2 = RandomForestRegressor(n_estimators=100, random_state=0)
eclf1 = VotingRegressor(estimators=[('lr', clf1), ('rf', clf2)]).fit(X, y)
eclf2 = skVotingRegressor(estimators=[('lr', clf1), ('rf', clf2)]).fit(X, y)
prob1 = eclf1.transform(X)
prob2 = eclf2.transform(X)
assert np.allclose(prob1, prob2)
pred1 = eclf1.predict(X)
pred2 = eclf2.predict(X)
assert np.allclose(pred1, pred2)